In this notebook, I validate my implementation of transformer components and GPT-2 small by:
- Comparing the behavior of a GPT-2 transformer block with my own implementation of a GPT-2 transformer block
- Comparing the behavior of the GPT-2 embedding components with my own implementation of the GPT-2 embedding components
- Comparing the behavior of the full GPT-2 model with my own implementation of the full GPT-2 model
- Sampling from my implementation of the full GPT-2 model

There are slight deviations between the outputs of my model and that of the original GPT-2 model. As shown below, this appears to be primarily due to the original GPT-2 model using a slightly different variant of GeLU from both me and PyTorch.

In [1]:
import torch as t
import sys
import os
notebook_path = os.path.abspath('')
project_root = os.path.join(notebook_path, '..')
sys.path.append(project_root)

## Comparing GPT2 transformer block (and components) with my implementation of a GPT2 transformer block

In [2]:
from transformers import GPT2Tokenizer, GPT2Model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2Model.from_pretrained('gpt2')

In [16]:
import torch as t
from src.blocks import TransformerDecoderBlock
from src.models import GPT2SmallModel


gpt2_block = model.h[0]
my_block = GPT2SmallModel().transformer_blocks[0]
#my_block = TransformerDecoderBlock(768, 12, 3072, 'gelu', 'layer_norm', 'mlpblock')

my_block.norm_layer1.weight, my_block.norm_layer1.bias = gpt2_block.ln_1.weight, gpt2_block.ln_1.bias
my_block.norm_layer2.weight, my_block.norm_layer2.bias = gpt2_block.ln_2.weight, gpt2_block.ln_2.bias

my_block.mlp_block.linear1.weight, my_block.mlp_block.linear1.bias = t.nn.Parameter(gpt2_block.mlp.c_fc.weight.T), gpt2_block.mlp.c_fc.bias
my_block.mlp_block.linear2.weight, my_block.mlp_block.linear2.bias = t.nn.Parameter(gpt2_block.mlp.c_proj.weight.T), gpt2_block.mlp.c_proj.bias

(wq,wk,wv) = t.chunk(gpt2_block.attn.c_attn.weight.T, 3, dim=0)
(bq,bk,bv) = t.chunk(gpt2_block.attn.c_attn.bias, 3, dim=0)

my_block.mha_block.linear_q.weight, my_block.mha_block.linear_q.bias = t.nn.Parameter(wq), t.nn.Parameter(bq)
my_block.mha_block.linear_k.weight, my_block.mha_block.linear_k.bias = t.nn.Parameter(wk), t.nn.Parameter(bk)
my_block.mha_block.linear_v.weight, my_block.mha_block.linear_v.bias = t.nn.Parameter(wv), t.nn.Parameter(bv)
my_block.mha_block.linear_o.weight, my_block.mha_block.linear_o.bias = t.nn.Parameter(gpt2_block.attn.c_proj.weight.T), gpt2_block.attn.c_proj.bias

my_block.eval()
gpt2_block.eval()

print(gpt2_block)
print(my_block)

GPT2Block(
  (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (attn): GPT2Attention(
    (c_attn): Conv1D()
    (c_proj): Conv1D()
    (attn_dropout): Dropout(p=0.1, inplace=False)
    (resid_dropout): Dropout(p=0.1, inplace=False)
  )
  (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (mlp): GPT2MLP(
    (c_fc): Conv1D()
    (c_proj): Conv1D()
    (act): NewGELUActivation()
    (dropout): Dropout(p=0.1, inplace=False)
  )
)
TransformerDecoderBlock(
  (norm_layer1): LayerNorm()
  (norm_layer2): LayerNorm()
  (mha_block): MultiheadAttentionBlock(
    (linear_q): Linear()
    (linear_k): Linear()
    (linear_v): Linear()
    (linear_o): Linear()
    (dropout_layer_1): Dropout()
    (dropout_layer_2): Dropout()
  )
  (mlp_block): MLPBlock(
    (linear1): Linear()
    (linear2): Linear()
    (dropout_layer): Dropout()
  )
)


#### Comparing transformer blocks

In [17]:
x = t.randn((1,5,768))

seq_len = x.shape[-2]
att_mask = t.where(t.arange(seq_len).unsqueeze(1) < t.arange(seq_len), -t.inf, 0)

gpt2_block.eval()
my_block.eval()

print("GPT2 Transformer block")
print(gpt2_block(x)[0])
print("My transformer block")
print(my_block(x))

GPT2 Transformer block
tensor([[[  7.6788,  -9.8812,  -1.1139,  ...,   0.0982,   0.7792,   6.4225],
         [ 14.1291,  -1.0713, -11.0099,  ...,  -5.2583,   3.5006,  -9.6592],
         [  7.8788,  -7.2983,  -9.1949,  ...,  -1.1361,   9.0922,  -0.3065],
         [ 11.2969,  -0.2765,  -1.4333,  ...,  28.7280,   0.3489,  21.2341],
         [ -6.0503, -11.5091, -14.0023,  ...,   5.4135,   0.0551,  12.2189]]],
       grad_fn=<AddBackward0>)
My transformer block
tensor([[[  7.6774,  -9.8810,  -1.1129,  ...,   0.0978,   0.7773,   6.4216],
         [ 14.1287,  -1.0719, -11.0098,  ...,  -5.2597,   3.5024,  -9.6582],
         [  7.8799,  -7.2980,  -9.1946,  ...,  -1.1365,   9.0916,  -0.3069],
         [ 11.2966,  -0.2760,  -1.4336,  ...,  28.7270,   0.3493,  21.2344],
         [ -6.0488, -11.5085, -14.0011,  ...,   5.4129,   0.0542,  12.2184]]],
       grad_fn=<AddBackward0>)


#### Comparing normalization

In [18]:
print("Normalization")

print("GPT2 Layer Norm 1")
print(gpt2_block.ln_1(x))
print("My Layer Norm 1")
print(my_block.norm_layer1(x))

print("GPT2 Layer Norm 2")
print(gpt2_block.ln_2(x))
print("My Layer Norm 2")
print(my_block.norm_layer2(x))

print(t.allclose(my_block.norm_layer1(x), gpt2_block.ln_1(x), atol=1e-7))
print(t.allclose(my_block.norm_layer2(x), gpt2_block.ln_2(x), atol=1e-7))


Normalization
GPT2 Layer Norm 1
tensor([[[-0.2543,  0.1051, -0.1672,  ..., -0.0284, -0.0908, -0.1497],
         [ 0.0776,  0.0596,  0.1473,  ..., -0.0333,  0.2456, -0.0011],
         [-0.2199, -0.0304,  0.0687,  ..., -0.1189,  0.2698,  0.2543],
         [ 0.3683, -0.4476,  0.1157,  ...,  0.2192, -0.1883, -0.2082],
         [-0.1518, -0.1926,  0.1469,  ...,  0.0816, -0.1222,  0.0852]]],
       grad_fn=<NativeLayerNormBackward0>)
My Layer Norm 1
tensor([[[-0.2543,  0.1051, -0.1672,  ..., -0.0284, -0.0908, -0.1497],
         [ 0.0776,  0.0596,  0.1473,  ..., -0.0333,  0.2456, -0.0011],
         [-0.2199, -0.0304,  0.0687,  ..., -0.1189,  0.2698,  0.2543],
         [ 0.3683, -0.4476,  0.1157,  ...,  0.2192, -0.1883, -0.2082],
         [-0.1518, -0.1926,  0.1469,  ...,  0.0816, -0.1222,  0.0852]]],
       grad_fn=<AddBackward0>)
GPT2 Layer Norm 2
tensor([[[-0.1046,  0.1222, -0.1344,  ..., -0.1356, -0.5330, -0.7598],
         [ 0.0902,  0.0700,  0.2890,  ..., -0.1691,  1.8359,  0.0809],
    

#### Comparing attention

In [22]:
print("Multi-head attention")
print("GPT2 Attention")
print(gpt2_block.attn(x)[0])
print("My attention")
print(my_block.mha_block(x, attention_mask=att_mask))


print(t.allclose(my_block.mha_block(x, attention_mask=att_mask), gpt2_block.attn(x)[0], atol=1e-4))

Multi-head attention
GPT2 Attention
tensor([[[ -6.8100,   4.8123,  -0.3440,  ...,  -0.1354,  -1.1235,   0.2230],
         [  2.7269,  -4.0638, -20.8347,  ...,  -0.1602,   0.6963,   1.3314],
         [-10.2985,  -9.5725,   0.0964,  ...,  -0.5052,   0.9192,  -0.1420],
         [ 11.7542,  -3.4020,   3.7599,  ...,  -1.3550,  -1.2914,   1.2410],
         [ -2.9027,  -8.9946,   1.8246,  ...,  -1.0319,   0.9347,   0.3394]]],
       grad_fn=<ViewBackward0>)
My attention
tensor([[[ -6.8100,   4.8123,  -0.3440,  ...,  -0.1354,  -1.1235,   0.2230],
         [  2.7269,  -4.0638, -20.8347,  ...,  -0.1602,   0.6963,   1.3314],
         [-10.2985,  -9.5725,   0.0964,  ...,  -0.5052,   0.9192,  -0.1420],
         [ 11.7542,  -3.4020,   3.7599,  ...,  -1.3550,  -1.2914,   1.2410],
         [ -2.9027,  -8.9945,   1.8246,  ...,  -1.0319,   0.9347,   0.3394]]],
       grad_fn=<AsStridedBackward0>)
True


#### Comparing MLP

In [26]:
print("MLP")
print("GPT2 MLP")
print(gpt2_block.mlp(x))
print("My MLP")
print(my_block.mlp_block(x))

print(t.allclose(my_block.mlp_block(x), gpt2_block.mlp(x), atol=1e-2))

MLP
GPT2 MLP
tensor([[[ 14.2362, -19.7986,  -0.4560,  ...,   9.6376,  13.1956,  14.2020],
         [ 14.4566,  -0.8697,  -4.6146,  ...,  -9.3027,  -1.3662, -13.2572],
         [  8.8456, -10.3722,  -9.0671,  ...,  -3.6412,  15.9834,  -1.3890],
         [  6.9620,  -2.3900,   0.5530,  ...,  45.6544,   9.9884,  23.9408],
         [ -6.6012, -11.3722, -19.0957,  ...,  12.6699,   0.8963,  13.0032]]],
       grad_fn=<ViewBackward0>)
My MLP
tensor([[[ 14.2347, -19.7987,  -0.4551,  ...,   9.6366,  13.1943,  14.2017],
         [ 14.4565,  -0.8689,  -4.6140,  ...,  -9.3020,  -1.3659, -13.2582],
         [  8.8456, -10.3701,  -9.0677,  ...,  -3.6403,  15.9842,  -1.3896],
         [  6.9620,  -2.3903,   0.5546,  ...,  45.6533,   9.9881,  23.9401],
         [ -6.6014, -11.3713, -19.0937,  ...,  12.6704,   0.8955,  13.0023]]],
       grad_fn=<AsStridedBackward0>)
True


#### Comparing GeLU

In [27]:
from src.activations import gelu
from torch.nn.functional import gelu as torch_gelu

print("GeLU")
x = t.randn((10,))

print("GPT2 GeLU")
print(gpt2_block.mlp.act(x))
print("My GeLU")
print(gelu(x))
print("Torch GeLU")
print(torch_gelu(x))


GeLU
GPT2 GeLU
tensor([-0.0454, -0.1560, -0.1437, -0.0401, -0.1681, -0.0629,  1.4251,  0.2870,
         0.0314, -0.1547])
My GeLU
tensor([-0.0455, -0.1560, -0.1435, -0.0401, -0.1680, -0.0629,  1.4253,  0.2870,
         0.0314, -0.1545])
Torch GeLU
tensor([-0.0455, -0.1560, -0.1435, -0.0401, -0.1680, -0.0629,  1.4253,  0.2870,
         0.0314, -0.1545])


#### Comparing MLP using GPT2 GeLU

In [28]:
print("My MLP with their GELU")
x = t.randn((1, 10, 768))

print("GPT2 MLP Block")
gpt2_mlp = gpt2_block.mlp(x)
print(gpt2_mlp)

print("My MLP Block with GPT2 GeLU")
my_mlp = my_block.mlp_block.linear2(gpt2_block.mlp.act(my_block.mlp_block.linear1(x)))
print(my_mlp)

print(t.allclose(gpt2_mlp, my_mlp, atol=1e-5))

print("The main difference between our models is in the MLP block, because OpenAI used a slightly different version of GeLU.")

My MLP with their GELU
GPT2 MLP Block
tensor([[[-34.8654,  -6.9767, -40.3958,  ...,  14.7084,  38.7968,   7.9544],
         [ -3.4955,  -6.0676,  19.7404,  ...,  21.4285,   1.5369,   9.7805],
         [ -0.1529,  -8.0439,  -7.7301,  ...,  -6.2543,   3.0814,   2.3790],
         ...,
         [ 17.7109,  -0.1405, -13.7513,  ...,   8.3319,  16.8486,  10.7691],
         [ 19.4083, -11.3437, -18.7254,  ...,   3.2344,   2.6756,  38.0403],
         [ -1.3154, -12.0614,  -5.8063,  ...,   0.9843,  -0.0852,  -6.5585]]],
       grad_fn=<ViewBackward0>)
My MLP Block with GPT2 GeLU
tensor([[[-34.8654,  -6.9767, -40.3958,  ...,  14.7084,  38.7968,   7.9544],
         [ -3.4955,  -6.0676,  19.7404,  ...,  21.4285,   1.5369,   9.7805],
         [ -0.1529,  -8.0439,  -7.7301,  ...,  -6.2543,   3.0814,   2.3790],
         ...,
         [ 17.7109,  -0.1405, -13.7513,  ...,   8.3319,  16.8486,  10.7691],
         [ 19.4083, -11.3437, -18.7254,  ...,   3.2344,   2.6756,  38.0403],
         [ -1.3154, -12.0

## Comparing GPT2 embedding with my implementation of the GPT2 embedding

In [29]:
from src.models import GPT2SmallModel
my_model = GPT2SmallModel()

from transformers import GPT2Tokenizer, GPT2Model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2Model.from_pretrained('gpt2')

my_model.embedding_layer.position_embedding_layer.weight = model.wpe.weight
my_model.embedding_layer.token_embedding_layer.weight = model.wte.weight

text = "This is a test sentence."
tokens = t.tensor(tokenizer(text)['input_ids']).unsqueeze(0)
tokens

tensor([[1212,  318,  257, 1332, 6827,   13]])

In [30]:
print("Embedding outputs")
print("GPT2 embedding output")
gpt2_emb_out = model.wte(tokens) + model.wpe(t.arange(tokens.shape[-1]))
print(gpt2_emb_out)
print("My embedding output")
my_emb_out = my_model.embedding_layer(tokens)
print(my_emb_out)

t.allclose(gpt2_emb_out, my_emb_out)

Embedding outputs
GPT2 embedding output
tensor([[[ 0.0065, -0.2930,  0.0762,  ...,  0.0184, -0.0275,  0.1638],
         [ 0.0142, -0.0437, -0.0393,  ...,  0.1487, -0.0278, -0.0255],
         [-0.0464, -0.0791,  0.1016,  ...,  0.0623,  0.0928, -0.0598],
         [-0.0580,  0.0095,  0.2207,  ..., -0.0635,  0.0760, -0.0543],
         [-0.0888, -0.0326,  0.1666,  ..., -0.2539, -0.0370, -0.2046],
         [ 0.0562, -0.0452,  0.1596,  ..., -0.0676,  0.0567,  0.0888]]],
       grad_fn=<AddBackward0>)
My embedding output
tensor([[[ 0.0065, -0.2930,  0.0762,  ...,  0.0184, -0.0275,  0.1638],
         [ 0.0142, -0.0437, -0.0393,  ...,  0.1487, -0.0278, -0.0255],
         [-0.0464, -0.0791,  0.1016,  ...,  0.0623,  0.0928, -0.0598],
         [-0.0580,  0.0095,  0.2207,  ..., -0.0635,  0.0760, -0.0543],
         [-0.0888, -0.0326,  0.1666,  ..., -0.2539, -0.0370, -0.2046],
         [ 0.0562, -0.0452,  0.1596,  ..., -0.0676,  0.0567,  0.0888]]],
       grad_fn=<AddBackward0>)


True

## Comparing GPT2 model vs. my implementation of the GPT2 model

In [31]:
from src.models import GPT2SmallModel
my_model = GPT2SmallModel()
my_model.eval()

# Importing GPT2LMHeadModel instead of GPT2Model because it applies the final unembed to get logits
from transformers import AutoTokenizer, GPT2LMHeadModel
tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
gpt2_model = GPT2LMHeadModel.from_pretrained("openai-community/gpt2")
gpt2_model.eval()

text = "This is a test sentence."
tokens = t.tensor(tokenizer(text)['input_ids']).unsqueeze(0)
tokens

tensor([[1212,  318,  257, 1332, 6827,   13]])

In [32]:
# Copying over weights
my_model.embedding_layer.position_embedding_layer.weight = gpt2_model.transformer.wpe.weight
my_model.embedding_layer.token_embedding_layer.weight = gpt2_model.transformer.wte.weight

my_model.final_ln.weight, my_model.final_ln.bias = gpt2_model.transformer.ln_f.weight, gpt2_model.transformer.ln_f.bias

my_model_block_list = my_model.transformer_blocks
gpt2_model_block_list = gpt2_model.transformer.h

for my_block, gpt2_block in zip(my_model_block_list, gpt2_model_block_list):
    my_block.norm_layer1.weight, my_block.norm_layer1.bias = gpt2_block.ln_1.weight, gpt2_block.ln_1.bias
    my_block.norm_layer2.weight, my_block.norm_layer2.bias = gpt2_block.ln_2.weight, gpt2_block.ln_2.bias

    my_block.mlp_block.linear1.weight, my_block.mlp_block.linear1.bias = t.nn.Parameter(gpt2_block.mlp.c_fc.weight.T), gpt2_block.mlp.c_fc.bias
    my_block.mlp_block.linear2.weight, my_block.mlp_block.linear2.bias = t.nn.Parameter(gpt2_block.mlp.c_proj.weight.T), gpt2_block.mlp.c_proj.bias

    (wq,wk,wv) = t.chunk(gpt2_block.attn.c_attn.weight.T, 3, dim=0)
    (bq,bk,bv) = t.chunk(gpt2_block.attn.c_attn.bias, 3, dim=0)

    my_block.mha_block.linear_q.weight, my_block.mha_block.linear_q.bias = t.nn.Parameter(wq), t.nn.Parameter(bq)
    my_block.mha_block.linear_k.weight, my_block.mha_block.linear_k.bias = t.nn.Parameter(wk), t.nn.Parameter(bk)
    my_block.mha_block.linear_v.weight, my_block.mha_block.linear_v.bias = t.nn.Parameter(wv), t.nn.Parameter(bv)
    my_block.mha_block.linear_o.weight, my_block.mha_block.linear_o.bias = t.nn.Parameter(gpt2_block.attn.c_proj.weight.T), gpt2_block.attn.c_proj.bias


### Logit outputs

In [33]:
print("Model logits")
print("GPT2 model logits")
gpt2_logits = gpt2_model(**tokenizer(text, return_tensors='pt')).logits
print(gpt2_logits)
print("My model logits")
my_logits = my_model(tokens)
print(my_logits)

print(t.allclose(gpt2_logits, my_logits, atol=1e-1))

Model logits
GPT2 model logits
tensor([[[ -35.8890,  -35.2049,  -39.1336,  ...,  -42.4869,  -41.8197,
           -36.0383],
         [-107.7291, -108.0175, -113.2967,  ..., -116.4645, -115.7443,
          -110.8654],
         [-111.7507, -111.5704, -114.5443,  ..., -120.7242, -117.1756,
          -112.3996],
         [ -86.1846,  -88.5057,  -94.3530,  ..., -101.3573,  -98.6974,
           -91.1616],
         [-106.4531, -108.7300, -115.4155,  ..., -119.6631, -119.1774,
          -110.7877],
         [-146.7139, -145.9828, -146.9487,  ..., -155.2113, -158.0557,
          -139.4035]]], grad_fn=<UnsafeViewBackward0>)
My model logits
tensor([[[ -35.8260,  -35.1460,  -39.0735,  ...,  -42.4222,  -41.7547,
           -35.9785],
         [-107.7155, -108.0055, -113.2770,  ..., -116.4519, -115.7324,
          -110.8482],
         [-111.7447, -111.5662, -114.5354,  ..., -120.7213, -117.1712,
          -112.3930],
         [ -86.1794,  -88.5035,  -94.3420,  ..., -101.3490,  -98.6924,
           -

### Argmax sampling

In [34]:
text = "The Large Apple, by Roald Dahl. \nLate at night,"
tokens_gpt2 = tokenizer(text, return_tensors='pt')
tokens_curr = tokenizer(text, return_tensors='pt')['input_ids']

tokens_gpt2

{'input_ids': tensor([[  464, 13601,  4196,    11,   416,  5564,  1940, 41471,    13,   220,
           198, 26302,   379,  1755,    11]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [35]:
for i in range(10):
    gpt2_next_token = gpt2_model(**tokens_gpt2).logits[0][-1].argmax()
    my_model_next_token = my_model(tokens_curr)[0][-1].argmax()

    tokens_gpt2['input_ids'] = t.cat([tokens_gpt2['input_ids'], gpt2_next_token.reshape((1,1))], dim=1)
    tokens_gpt2['attention_mask'] = t.cat([tokens_gpt2['attention_mask'], t.tensor(1).reshape((1,1))], dim=1)
    tokens_curr = t.cat([tokens_curr, my_model_next_token.reshape((1,1))], dim=1)

In [36]:
print("GPT2 argmax output")
print(''.join(tokenizer.convert_ids_to_tokens(tokens_gpt2['input_ids'][0])).replace('Ġ', ' ').replace('Ċ', '\n'))
print()
print("My implementation output")
print(''.join(tokenizer.convert_ids_to_tokens(tokens_curr[0])).replace('Ġ', ' ').replace('Ċ', '\n'))

GPT2 argmax output
The Large Apple, by Roald Dahl. 
Late at night, I was sitting in the living room of my home

My implementation output
The Large Apple, by Roald Dahl. 
Late at night, I was sitting in the living room of my home


## Normal sampling from my version of the GPT-2 model

In [40]:
# https://openai.com/index/better-language-models/
text = "In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English."
tokens = tokenizer(text, return_tensors='pt')['input_ids']

tokens

tensor([[  818,   257, 14702,  4917,    11, 11444,  5071,   257, 27638,   286,
         28000, 19942,  2877,   287,   257,  6569,    11,  4271, 31286,  1850,
         19272,    11,   287,   262,   843,   274, 21124,    13,  3412,   517,
          6452,   284,   262,  4837,   373,   262,  1109,   326,   262, 28000,
         19942,  5158,  2818,  3594,    13]])

In [41]:
for i in range(100):
    my_model_next_token = t.multinomial(t.softmax(my_model(tokens)[0][-1], dim=0), 1)

    tokens = t.cat([tokens, my_model_next_token.unsqueeze(0)], dim=1)

In [39]:
print("GPT2 completion of unicorn prompt")
print(''.join(tokenizer.convert_ids_to_tokens(tokens[0])).replace('Ġ', ' ').replace('Ċ', '\n'))

GPT2 completion of unicorn prompt
In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English. This clearly opens the door to potential insights into human language.

Warwicz had his humble beginnings in the fall of 1948 as a visiting missionary at Judeo-Christian monastery in Guatemala City, growing up in the tightly knit neighborhoods of Bolivian refugee camps, living close to his ancestral families, where his desire to learn had never been burned. Lacking a university education, he passed Jeremiah B. McCaw in segregation and Catholic school records, and was the youngest of three children. He
